In [1]:
import numpy as np

import adaptoctree.morton as morton
import adaptoctree.tree as tree
import adaptoctree.plotting as plotting

# Test logic

In [80]:
N = int(1e3)
particles = plotting.make_moon(N)
# particles = np.random.rand(N, 3)

max_level = 16
max_num_particles = 150

In [81]:
max_bound, min_bound = morton.find_bounds(particles)
x0 = morton.find_center(max_bound, min_bound)
r0 = morton.find_radius(x0, max_bound, min_bound)

In [82]:
unbalanced = tree.build(particles, max_level, max_num_particles, 1)
tst = unbalanced
unbalanced = np.unique(unbalanced)
depth = max(morton.find_level(unbalanced))

In [83]:
len(unbalanced)

12

In [84]:
balanced = tree.balance(unbalanced, depth)

In [85]:
balanced = np.fromiter(balanced, np.int64, len(balanced))

In [86]:
len(balanced)

59

In [87]:
balanced_leaves = tree.assign_points_to_keys(particles, balanced, x0, r0)

In [88]:
point = particles[balanced_leaves == -1][0]

In [89]:
particles[balanced_leaves == -1]

array([[7.15517514, 0.82305275, 1.42274953]])

In [62]:
key = morton.encode_point(point, 3, x0, r0)

In [73]:
key

7471107

In [63]:
morton.decode_key(key)

array([4, 4, 3, 3], dtype=int32)

In [64]:
set(morton.find_descendents(key, 1)).intersection(balanced)a#

{59801604}

In [65]:
morton.decode_key(key)

array([4, 4, 3, 3], dtype=int32)

In [66]:
key in set(balanced)

False

In [74]:
morton.decode_key(morton.find_parent(key))

array([2, 2, 1, 2], dtype=int32)

In [75]:
morton.find_children(morton.find_parent(key))

array([7340035, 7372803, 7405571, 7438339, 7471107, 7503875, 7536643,
       7569411])

In [76]:
key in set(unbalanced)

True

In [35]:
balanced.shape

(380,)

In [100]:
# Test num particles constraint
_, counts = np.unique(balanced_leaves, return_counts=True)

In [101]:
assert np.all(counts <= max_num_particles)

AssertionError: 

In [12]:
counts < max_num_particles

array([ True])

In [97]:
# Check for balancing condition

for i in balanced:
    for j in balanced:
        if (i != j) and morton.are_neighbours(i, j, x0, r0):
            diff = morton.find_level(i) - morton.find_level(j)
            assert diff <= 1

In [99]:
# Check for overlaps

for i, ki in enumerate(balanced):
    for j, kj in enumerate(balanced):
        if j != i:
            assert ki not in morton.find_ancestors(kj)            

# Benchmarking

In [ ]:
N = int(1e5)
particles = plotting.make_moon(N)
# particles = np.random.rand(N, 3)

max_level = 16
max_num_particles = 5


max_bound, min_bound = morton.find_bounds(particles)
x0 = morton.find_center(max_bound, min_bound)
r0 = morton.find_radius(x0, max_bound, min_bound)

unbalanced = tree.build(particles, max_level=max_level, max_points=5)
unbalanced, counts = np.unique(unbalanced, return_counts=True)

len(unbalanced)

In [ ]:
unbalanced

In [ ]:
%timeit tree.build(particles, max_level, max_num_particles)

In [ ]:
%timeit tree.balance(unbalanced, depth)

In [ ]:
tst = tree.balance(unbalanced, depth)

In [ ]:
tst == set(unbalanced)

In [ ]:
import numba as nb

reflected_int_set = nb.types.Set(nb.int64, reflected=False)

@nb.njit(reflected_int_set(reflected_int_set, reflected_int_set))
def f(set_1, set_2):
    return set_1
